# AI Podcast Analysis

This notebook analyzes 1.7MB of personal podcast listening data exported from Snipd, an app that generates AI notes for interesting podcast moments. The analysis explores:

Listening patterns and metadata (timestamps, show distribution, snip metrics)
Content themes using NLP (keyword extraction, topic modeling, similarity mapping)
Political sentiment analysis using LLMs

Data format: Markdown files containing episode metadata, AI-generated notes, and timestamped snippets with transcripts.
Purpose: Understand podcast consumption habits, content preferences, and potential influence on political perspectives.

## 1) Data Ingestion and Parsing

1. **File Parsing**: Read the `.md` file line by line or segment by segment, and extract:
   - Episode title, show name, date
   - Link(s)
   - Timestamps for snips
   - The actual text transcripts/notes
2. **Data Structure**: Store everything in a structured format like a Pandas DataFrame:
   ```python
   df = pd.DataFrame(columns=[
       'show_name', 'episode_title', 'snip_text', 'snip_timestamp', 
       'episode_date', 'other_metadata'
   ])
   ```

In [147]:
import re
def parse_markdown_to_dataframe(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split content into episodes based on top-level headings (# )
    episode_blocks = re.split(r'(?m)^# ', content)
    episode_blocks = [block.strip() for block in episode_blocks if block.strip()]
    
    all_snips = []

    for idx, block in enumerate(episode_blocks):
        print(f"Processing Episode {idx + 1}")
        block = '# ' + block
        print(block)
        break


parse_markdown_to_dataframe('snipd_export_2024-12-24_14-55.md')

Processing Episode 1
# #393 - Is History Repeating Itself?


<img src="https://wsrv.nl/?url=https%3A%2F%2Fassets.samharris.org%2Fimages%2Frss%2Fmaking-sense-logo.png&w=200&h=200" width="200" alt="Cover" />


## Episode metadata
- Episode title: #393 - Is History Repeating Itself?
- Show: Making Sense with Sam Harris - Subscriber Content
- Owner / Host: Making Sense with Sam Harris
- Episode link: [open in Snipd](https://share.snipd.com/episode/ba69c343-dda0-4730-9e73-41bb86a19ad9)
- Episode publish date: 2024-11-26
<details>
<summary>Show notes</summary>
> Share this episode:  https://www.samharris.org/podcasts/making-sense-episodes/393-is-history-repeating-itself <br/>> <br/>>  Sam Harris speaks with Simon Sebag Montefiore about the ongoing conflict in the Middle East, the history of the Jews, and the rise of global antisemitism.<br/>> <br/>>   Simon Sebag Montefiore  is an internationally bestselling author and historian. His books include   Catherine the Great and Potemkin  ,   Stal

In [53]:
import re
def parse_markdown_to_dataframe(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split content into episodes based on top-level headings (# )
    episode_blocks = re.split(r'(?m)^# ', content)
    episode_blocks = [block.strip() for block in episode_blocks if block.strip()]
    
    all_snips = []

    for idx, block in enumerate(episode_blocks):
        print(f"Processing Episode {idx + 1}")
        block = '# ' + block

        # Extract Episode Metadata
        episode_title_match = re.search(r'- Episode title:\s*(.*)', block)
        show_match = re.search(r'- Show:\s*(.*)', block)
        owner_host_match = re.search(r'- Owner / Host:\s*(.*)', block)
        episode_link_match = re.search(r'- Episode link:\s*\[.*?\]\((.*?)\)', block)
        publish_date_match = re.search(r'- Episode publish date:\s*(.*)', block)
        export_date_match = re.search(r'- Export date:\s*(.*)', block)

        episode_title = episode_title_match.group(1).strip() if episode_title_match else ''
        show = show_match.group(1).strip() if show_match else ''
        owner_host = owner_host_match.group(1).strip() if owner_host_match else ''
        episode_link = episode_link_match.group(1).strip() if episode_link_match else ''
        publish_date = publish_date_match.group(1).strip() if publish_date_match else ''
        export_date = export_date_match.group(1).strip() if export_date_match else ''

        print(f"Episode Title: {episode_title}")
        print(f"Show: {show}")
        print(f"Owner/Host: {owner_host}")
        print(f"Episode Link: {episode_link}")
        print(f"Publish Date: {publish_date}")
        print(f"Export Date: {export_date}")

        # Extract Snips Section
        snips_section_match = re.search(r'## Snips(.*?)(?=\n## |\Z)', block, re.DOTALL)
        if snips_section_match:
            snips_section = snips_section_match.group(1).strip()
            print(f"Found Snips Section with length {len(snips_section)} characters.")
        else:
            snips_section = ''
            print("No Snips Section found.")

        # Split snips based on '---' separators
        snip_blocks = re.split(r'\n-{3,}\n', snips_section)
        print(f"Found {len(snip_blocks)} snips in this episode.")

        for snip_idx, snip_block in enumerate(snip_blocks):
            snip_block = snip_block.strip()
            if not snip_block:
                continue  # Skip empty snip blocks

            # Extract Timestamp and Title
            snip_header_match = re.match(r'### \[(\d{2}:\d{2})\] (.+)', snip_block)
            if not snip_header_match:
                print(f"  Snip {snip_idx + 1}: No snip header match found.")
                continue  # Skip if no match
            snip_timestamp, snip_title = snip_header_match.groups()
            snip_title = snip_title.strip()
            print(f"  Snip {snip_idx + 1}: Timestamp: {snip_timestamp}, Title: {snip_title}")

            # Extract Play Link, Duration, and Time Range
            play_link_match = re.search(r'\[🎧 Play snip - ([^\)]+)\(([^)]+)\)\]\((.*?)\)', snip_block)
            if play_link_match:
                duration = play_link_match.group(1).strip()
                time_range = play_link_match.group(2).strip()
                play_link = play_link_match.group(3).strip()
                print(f"    Play Link: {play_link}, Duration: {duration}, Time Range: {time_range}")
            else:
                duration = ''
                time_range = ''
                play_link = ''
                print(f"    No Play Link found.")

            # Extract Summary
            summary_match = re.search(r'### ✨ Summary\s*\n(.*?)\n-{3,}', snip_block, re.DOTALL)
            if not summary_match:
                summary_match = re.search(r'### ✨ Summary\s*\n(.*)', snip_block, re.DOTALL)
            summary = summary_match.group(1).strip() if summary_match else ''
            if summary:
                print(f"    Summary extracted.")
            else:
                print(f"    No Summary found.")

            # Extract Transcript
            transcript_match = re.search(r'#### 📚 Transcript\s*\n<details>.*?(<blockquote>.*?</blockquote>).*?</details>', snip_block, re.DOTALL)
            if transcript_match:
                transcripts = re.findall(r'<blockquote><b>(.*?)</b><br/><br/>(.*?)</blockquote>', snip_block, re.DOTALL)
                transcript_text = "\n".join([f"{speaker.strip()}: {text.strip()}" for speaker, text in transcripts])
                print(f"    Transcript extracted with {len(transcripts)} speakers.")
            else:
                transcript_text = ''
                print(f"    No Transcript found.")

            # Compile all data into a dictionary
            snip_data = {
                'episode_title': episode_title,
                'show': show,
                'owner_host': owner_host,
                'episode_link': episode_link,
                'publish_date': publish_date,
                'export_date': export_date,
                'snip_timestamp': snip_timestamp,
                'snip_title': snip_title,
                'play_link': play_link,
                'duration': duration,
                'time_range': time_range,
                'summary': summary,
                'transcript': transcript_text,
                'full_episode_md': block
            }

            all_snips.append(snip_data)

    # Create DataFrame
    df = pd.DataFrame(all_snips)
    return df



# File path
file_path = 'snipd_export_2024-12-24_14-55.md'

# Parse the markdown file into a DataFrame
df = parse_markdown_to_dataframe(file_path)

# Display the DataFrame
print(df.head())



Processing Episode 1
Episode Title: #393 - Is History Repeating Itself?
Show: Making Sense with Sam Harris - Subscriber Content
Owner/Host: Making Sense with Sam Harris
Episode Link: https://share.snipd.com/episode/ba69c343-dda0-4730-9e73-41bb86a19ad9
Publish Date: 2024-11-26
Export Date: 2024-12-24T14:55
Found Snips Section with length 29167 characters.
Found 11 snips in this episode.
  Snip 1: Timestamp: 08:00, Title: Exceptional Period of Stability Ending
    Play Link: https://share.snipd.com/snip/fe100138-429c-4fe7-90fe-d78187e98c6f, Duration: 2min️, Time Range: 05:46 - 08:05
    No Summary found.
    Transcript extracted with 3 speakers.
  Snip 2: Timestamp: 41:21, Title: Origin of Antisemitism
    Play Link: https://share.snipd.com/snip/54a1ffb1-1231-4623-8d48-62bb190dcee4, Duration: 2min️, Time Range: 39:50 - 41:21
    No Summary found.
    Transcript extracted with 1 speakers.
  Snip 3: Timestamp: 44:50, Title: Jewish Otherness
    Play Link: https://share.snipd.com/snip/2b917

In [142]:
len(df['episode_title'].unique())

181

In [64]:
# find rows with missing transcript or summary
missing_data = df[(df['transcript'].str.len() < 10) | (df['summary'].str.len() < 10)]
missing_data


,episode_title,show,owner_host,episode_link,publish_date,export_date,snip_timestamp,snip_title,play_link,duration,time_range,summary,transcript,full_episode_md
0,#393 - Is History Repeating Itself?,Making Sense with Sam Harris - Subscriber Content,Making Sense with Sam Harris,https://share.snipd.com/episode/ba69c343-dda0-...,2024-11-26,2024-12-24T14:55,08:00,Exceptional Period of Stability Ending,https://share.snipd.com/snip/fe100138-429c-4fe...,2min️,05:46 - 08:05,,Simon Seabag Montefiore: But I think the thing...,# #393 - Is History Repeating Itself?\n\n\n<im...
1,#393 - Is History Repeating Itself?,Making Sense with Sam Harris - Subscriber Content,Making Sense with Sam Harris,https://share.snipd.com/episode/ba69c343-dda0-...,2024-11-26,2024-12-24T14:55,41:21,Origin of Antisemitism,https://share.snipd.com/snip/54a1ffb1-1231-462...,2min️,39:50 - 41:21,,"Simon Seabag Montefiore: But you're right, fro...",# #393 - Is History Repeating Itself?\n\n\n<im...
2,#393 - Is History Repeating Itself?,Making Sense with Sam Harris - Subscriber Content,Making Sense with Sam Harris,https://share.snipd.com/episode/ba69c343-dda0-...,2024-11-26,2024-12-24T14:55,44:50,Jewish Otherness,https://share.snipd.com/snip/2b917d9c-dd6f-459...,1min️,43:34 - 44:48,,Simon Seabag Montefiore: And I think it was be...,# #393 - Is History Repeating Itself?\n\n\n<im...
3,#393 - Is History Repeating Itself?,Making Sense with Sam Harris - Subscriber Content,Making Sense with Sam Harris,https://share.snipd.com/episode/ba69c343-dda0-...,2024-11-26,2024-12-24T14:55,49:42,Crusader Massacre in Jerusalem,https://share.snipd.com/snip/319c8640-bdaf-436...,1min️,48:17 - 49:42,,Simon Seabag Montefiore: And then they fought ...,# #393 - Is History Repeating Itself?\n\n\n<im...
4,#393 - Is History Repeating Itself?,Making Sense with Sam Harris - Subscriber Content,Making Sense with Sam Harris,https://share.snipd.com/episode/ba69c343-dda0-...,2024-11-26,2024-12-24T14:55,51:15,The Crusades and October 7th,https://share.snipd.com/snip/30d1e5d0-b9ae-44d...,1min️,49:47 - 51:14,,Simon Seabag Montefiore: So a chilling moment....,# #393 - Is History Repeating Itself?\n\n\n<im...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,"Warum gibt die EU so ein schwaches Bild ab, Ma...",The Pioneer Briefing,Gabor Steingart,https://share.snipd.com/episode/5c15966b-cf73-...,2024-03-21,2024-12-24T14:55,06:47,Untitled,https://share.snipd.com/snip/5f4c39fc-1336-437...,1min️,05:27 - 06:47,,,"# Warum gibt die EU so ein schwaches Bild ab, ..."
398,#364 – Chris Voss: FBI Hostage Negotiator,Lex Fridman Podcast,Lex Fridman,https://share.snipd.com/episode/afb1e4cc-e91d-...,2023-03-10,2024-12-24T14:55,16:42,Articulate the Perspective of Others,https://share.snipd.com/snip/5a706cfd-8c9f-4dd...,3min️,15:25 - 18:52,Empathy is the ability to understand and artic...,,# #364 – Chris Voss: FBI Hostage Negotiator\n\...
399,#280 — The Future of Artificial Intelligence,Making Sense with Sam Harris,Waking Up with Sam Harris,https://share.snipd.com/episode/3d91835a-9b48-...,2022-04-22,2024-12-24T14:55,34:49,The potential Danger in Future in AI,https://share.snipd.com/snip/59b9fb7e-d865-4ff...,6min️,28:37 - 34:49,The development of artificial intelligence (AI...,,# #280 — The Future of Artificial Intelligence...
400,#57 David Deutsch - The Multiverse is Real,Within Reason,Alex O'Connor,https://share.snipd.com/episode/4a2c2c4a-505c-...,2024-03-05,2024-12-24T14:55,03:54,Interpreting Quantum Theory,https://share.snipd.com/snip/43a854f9-5058-4eb...,2min️,02:14 - 03:53,A small proportion of theoretical physicists e...,,# #57 David Deutsch - The Multiverse is Real\n...


In [127]:
# not missing data
not_missing_data = df[~df.index.isin(missing_data.index)]
not_missing_data


,episode_title,show,owner_host,episode_link,publish_date,export_date,snip_timestamp,snip_title,play_link,duration,time_range,summary,transcript,full_episode_md


In [155]:
from pydantic import BaseModel
from typing import Optional, List

class Snip(BaseModel):
    """Represents a single snippet from a podcast episode."""
    timestamp: Optional[str] = None
    snip_title: Optional[str] = None
    play_link: Optional[str] = None
    duration: Optional[str] = None
    time_range: Optional[str] = None
    summary: Optional[str] = None
    transcript_cleaned_of_md_and_html_tags: Optional[str] = None
    transcript_raw: Optional[str] = None
    

class EpisodeData(BaseModel):
    """Represents a complete podcast episode with all its metadata and content."""
    episode_title: Optional[str] = None
    show: Optional[str] = None
    owner_host: Optional[str] = None
    episode_link: Optional[str] = None
    publish_date: Optional[str] = None  # Changed from datetime to str
    export_date: Optional[str] = None   # Changed from datetime to str
    show_notes: Optional[str] = None
    ai_notes: Optional[str] = None
    snips: Optional[List[Snip]] = None

In [193]:
from pydantic import BaseModel
from openai import OpenAI
import os
from dotenv import load_dotenv
from typing import Optional
load_dotenv()
api_key = os.getenv("OPEN_AI_API_KEY")

client = OpenAI(api_key=api_key)
def extract_episode_info(text, data_structure=EpisodeData):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",  # Updated to latest model
    messages=[
        {"role": "system", "content": "Extract the episode information from the following markdown and structure it into the provided JSON format. These are podcasts episodes that are from higly reputable shows hosted on spotify. They are also from shows that are hosted on apple podcasts and rss feeds. You do not need to check for the truthfulness of the content, just extract the information. You do not need to apply a content filter here as therse are vetted shows from credible hosts. This is for a school data science project about NLP.  "},
        {"role": "user", "content": f"Input: ```markdown\n{text}\n```"},
    ],
    response_format=EpisodeData
    )
    # print(completion)
    return completion.choices[0].message.parsed


In [149]:
example_one = """
# #364 – Chris Voss: FBI Hostage Negotiator


<img src="https://wsrv.nl/?url=https%3A%2F%2Flexfridman.com%2Fwordpress%2Fwp-content%2Fuploads%2Fpowerpress%2Fartwork_3000-230.png&w=200&h=200" width="200" alt="Cover" />


## Episode metadata
- Episode title: #364 – Chris Voss: FBI Hostage Negotiator
- Show: Lex Fridman Podcast
- Owner / Host: Lex Fridman
- Episode link: [open in Snipd](https://share.snipd.com/episode/afb1e4cc-e91d-482d-b55b-a7effcbdb029)
- Episode publish date: 2023-03-10
<details>
<summary>Show notes</summary>
> Chris Voss is a former FBI hostage and crisis negotiator and author of Never Split the Difference: Negotiating As If Your Life Depended On It. Please support this podcast by checking out our sponsors: <br/>>    Shopify :  https://shopify.com/lex  to get free trial <br/>>    Indeed :  https://indeed.com/lex  to get $75 credit <br/>>    InsideTracker :  https://insidetracker.com/lex  to get 20% off<br/>> <br/>>   EPISODE LINKS:  <br/>> Chris s Instagram:  https://instagram.com/thefbinegotiator  <br/>> Chris s Twitter:  https://twitter.com/fbinegotiator  <br/>> Chris s Website:  https://blackswanltd.com  <br/>> Chris s Masterclass:  https://masterclass.com/classes/chris-voss-teaches-the-art-of-negotiation  <br/>> Never Split the Difference (book):  https://amzn.to/3J5scNC <br/>> <br/>>   PODCAST INFO:  <br/>> Podcast website:  https://lexfridman.com/podcast  <br/>> Apple Podcasts:  https://apple.co/2lwqZIr  <br/>> Spotify:  https://spoti.fi/2nEwCF8  <br/>> RSS:  https://lexfridman.com/feed/podcast/  <br/>> YouTube Full Episodes:  https://youtube.com/lexfridman  <br/>> YouTube Clips:  https://youtube.com/lexclips <br/>> <br/>>   SUPPORT   Check out the sponsors above, it s the best way to support this podcast <br/>>   Support on Patreon:  https://www.patreon.com/lexfridman  <br/>>   Twitter:  https://twitter.com/lexfridman  <br/>>   Instagram:  https://www.instagram.com/lexfridman  <br/>>   LinkedIn:  https://www.linkedin.com/in/lexfridman  <br/>>   Facebook:  https://www.facebook.com/lexfridman  <br/>>   Medium:  https://medium.com/@lexfridman <br/>> <br/>>   OUTLINE:  <br/>> Here s the timestamps for the episode. On some podcast players you should be able to click the timestamp to jump to that time. <br/>> (00:00)   Introduction <br/>> (06:31)   Negotiation <br/>> (12:21)   Reason vs Emotion <br/>> (27:17)   How to listen <br/>> (36:06)   Negotiation with terrorists <br/>> (38:14)   Brittney Griner <br/>> (39:53)   Putin and Zelenskyy <br/>> (47:13)   Donald Trump <br/>> (54:23)   When to walk away <br/>> (58:37)   Israel and Palestine <br/>> (1:06:16)   Al-Qaeda <br/>> (1:11:46)   Three voices of negotiation <br/>> (1:20:11)   Strategic umbrage <br/>> (1:23:18)   Mirroring <br/>> (1:26:29)   Labeling <br/>> (1:33:55)   Exhaustion <br/>> (1:36:09)   The word  fair  <br/>> (1:39:06)   Closing the deal <br/>> (1:41:03)   Manipulation and lying <br/>> (1:42:58)   Conversation vs Negotiation <br/>> (1:54:17)   The 7-38-55 Rule <br/>> (1:58:16)   Chatbots <br/>> (2:07:39)   War <br/>> (2:09:10)   Advice for young people
</details>

- Show notes link: [open website](https://lexfridman.com/chris-voss/?utm_source=rss&utm_medium=rss&utm_campaign=chris-voss)
- Export date: 2024-12-24T14:55


## Snips


### [16:42] Articulate the Perspective of Others


[🎧 Play snip - 3min️ (15:25 - 18:52)](https://share.snipd.com/snip/5a706cfd-8c9f-4dd9-ac73-a0880c9fdc1a)




### ✨ Summary
Empathy is the ability to understand and articulate the perspective of others without necessarily agreeing or liking them. It requires straight understanding of where they're coming from, without the need for agreement. Empathy can be a powerful tool as it allows you to connect with and understand anyone, even if you do not sympathize or agree with them. It is important to show that you understand someone's perspective, regardless of your own views, in order to have a meaningful conversation and potentially bridge gaps of understanding.


---




#### 📚 Transcript
<details>
<summary>Click to expand</summary>
<blockquote><b>Speaker 1</b><br/><br/>So Bob's definition of empathy said not agreeing or even liking the other side. Don't even got to like them. Don't got to agree with them. Just straight understanding where they're coming from and articulating it, which requires no agreement whatsoever. That becomes a very powerful tool, like ridiculously powerful. And if sympathy or compassion or agreement are not included, you can be empathic with anybody. I was thinking about this when I was getting ready to sit down and talk to you because you use the word empathy a lot. Putin. I can be empathic with Putin. Easy. It's easy. I don't agree with where he's coming from. I don't agree with his methodology. Only on the Ukraine-Russian war, I saw an article that was very dismissive of Russia that said, Russia's basically Europe's gas station. And I thought, all right. So if you're in charge and the way you feed your people is via an industry that the entire world is trying to quit, the whole world is trying to get out of fossil fuels. But that's how you feed your people. If you don't come up with an answer to that, the people that you've taken a responsibility for are going to die alone in the cold and the dark. They're going to freeze and they're going to die. All right. So that doesn't mean that I agree with where he's coming from or any of his means. But where is, how does this guy see things? It is a distorted word. You're never going to get through to somebody like that in a conversation unless you can demonstrate to them you understand where they're coming from, whether or not you agree. In the early 90s, last century, I'm a last century guy. I'm an old dude. Refer to myself as a last century guy, also a deeply flawed human. So terrorist case, New York City, civilian court, terrorism does not have to be tried in military tribunals. That's a very bad idea. It was always bad. The FBI was always against it. I'm getting ready, we have Muslims testifying in open court against the legitimate Muslim cleric, the guy that was on trial had the credentials as a legitimate Muslim cleric. The people that were testifying against him didn't think he should be advocating murder of innocent people. We'd sit down with them, Arab Muslims, Egyptians, mostly. And I would say to them, you believe that there's been a succession of American governments for the last 200 years that are anti-Islamic and they shake their head and go, yeah. And that'd be the start of the conversation. That's empathy. You believe this to be the case. I never said I agreed. I never said I disagreed. But I showed them that I wasn't afraid of their beliefs. I was so unafraid of them that I was willing to just state them and not disagree or contradict because I would say that and then I'd shut up and let them react. And I never had to say, here's why you're wrong. I never gave my point of view. Every single one of them are testified. That's empathy, not agreeing where the other side is coming from.</blockquote>
</details>



---

"""

example_two = """

# New SEC Chair, Bitcoin, xAI Supercomputer, UnitedHealth CEO murder, with Gavin Baker & Joe Lonsdale


<img src="https://wsrv.nl/?url=https%3A%2F%2Fstatic.libsyn.com%2Fp%2Fassets%2Fa%2F9%2Fc%2Fb%2Fa9cb4d1dadb1ea21%2Fall-in_logo.png&w=200&h=200" width="200" alt="Cover" />


## Episode metadata
- Episode title: New SEC Chair, Bitcoin, xAI Supercomputer, UnitedHealth CEO murder, with Gavin Baker & Joe Lonsdale
- Show: All-In with Chamath, Jason, Sacks & Friedberg
- Owner / Host: All-In Podcast, LLC
- Episode link: [open in Snipd](https://share.snipd.com/episode/fa104938-398c-4e37-92e7-a0d75de82124)
- Episode publish date: 2024-12-07
<details>
<summary>Show notes</summary>
> (0:00) Bestie announcement!<br/>>   (2:53) Gavin Baker and Joe Lonsdale join the show<br/>>   (4:14) State of the Trump Bump: Debt focus, Deregulation, America's lucky position<br/>>   (20:08) Trump nominates Paul Atkins as SEC Chair, replacing Gary Gensler: What this means for crypto and other markets<br/>>   (41:07) Thoughts on Michael Saylor's Bitcoin play, state of defense tech, and the US/China AI competition<br/>>   (49:25) xAI's massive GPU cluster, expanding to 1M GPUs, how Grok 3 will test AI scaling laws, and what's next<br/>>   (1:08:28) UnitedHealth CEO murdered, reactions<br/>>   Get virtual tickets for The All-In Holiday Spectacular!:<br/>>    https://allin.com/events <br/>>   Follow the besties:<br/>>    https://x.com/chamath <br/>>    https://x.com/Jason <br/>>    https://x.com/DavidSacks <br/>>    https://x.com/friedberg <br/>>   Follow Gavin Baker:<br/>>    https://x.com/GavinSBaker <br/>>   Follow Joe Lonsdale:<br/>>    https://x.com/JTLonsdale <br/>>   Follow on X:<br/>>    https://x.com/theallinpod <br/>>   Follow on Instagram:<br/>>    https://www.instagram.com/theallinpod <br/>>   Follow on TikTok:<br/>>    https://www.tiktok.com/@theallinpod <br/>>   Follow on LinkedIn:<br/>>    https://www.linkedin.com/company/allinpod <br/>>   Intro Music Credit:<br/>>    https://rb.gy/tppkzl <br/>>    https://x.com/yung_spielburg <br/>>   Intro Video Credit:<br/>>    https://x.com/TheZachEffect <br/>>   Referenced in the show:<br/>>     https://truthsocial.com/@realDonaldTrump/posts/113603133222686186 <br/>>     https://www.nytimes.com/2024/12/04/business/trump-sec-paul-atkins.html <br/>>    https://x.com/davidmarcus/status/1862654506774810641 <br/>>     https://www.bloomberg.com/news/articles/2024-12-05/convertible-bond-arbs-are-making-microstrategy-wall-street-s-hottest-trade <br/>>    https://www.ft.com/content/9c0516cf-dd12-4665-aa22-712de854fe2f <br/>>     https://www.nytimes.com/live/2024/12/04/nyregion/brian-thompson-uhc-ceo-shot <br/>>     https://abcnews.go.com/US/man-shot-chest-midtown-manhattan-masked-gunman-large/story?id=116446382&cid=social_twitter_abcn <br/>>     https://nypost.com/2024/12/06/media/taylor-lorenz-defends-unitedhealthcare-ceo-brian-thompson-jokes
</details>

- Show notes link: [open website](https://sites.libsyn.com/254861/new-sec-chair-bitcoin-xai-supercomputer-unitedhealth-ceo-murder-with-gavin-baker-joe-lonsdale)
- Export date: 2024-12-24T14:55


## Snips


### [06:16] America's Untapped Potential


[🎧 Play snip - 2min️ (04:43 - 06:15)](https://share.snipd.com/snip/fa8a6dd3-4d74-4136-b86f-c117301884e5)


**America's Untapped Potential**

- America, like Microsoft under Ballmer, has immense potential but has been mismanaged.
- By simply stopping "really, really dumb things" like excessive regulations, America can unlock significant growth.
- This is analogous to Satya Nadella's turnaround of Microsoft, which involved ceasing counterproductive practices.
- A key area for improvement is reducing excessive regulations that hinder development.
-  This has led to too many administrators and unnecessary complexities, exemplified by the inefficient allocation of funds for rural broadband and EV chargers under the Harris administration.


#### 📚 Transcript
<details>
<summary>Click to expand</summary>
<blockquote><b>Gavin Baker</b><br/><br/>So if they execute unstated plans and there are some of the world's greatest execution machines involved, Elon generally does what he says he's going to do. Like, this is going to be awesome for America, for markets, for the world. And the analogy I keep coming back to is Satya Nadella taking over as CEO of Microsoft. Microsoft was a monopoly, incredibly advantaged. It had just been horribly mismanaged for years. All he had to do to start winning was stop doing really, really dumb things. And that's an incredible place to be. You know, America, like we're the greatest country. You know, we've got, you know, oceans on two sides, peaceful neighbors, incredible natural resources, you know, completely, you know, can produce our own food and energy, like in Many ways, most privileged country on earth. But sometimes with great privilege comes great, like stupidity. And California, to me, would be a leading example of that. Most, in many ways, most privileged state in America, and has printed it away with bad policies. And I do think one thing that everyone of all political stripes agrees on is there are too many regulations that result in far too many administrators, far too much complexity, and an Inability to build things in America. So, you know, it was used very effectively,</blockquote>
</details>



---


### [17:16] Nuclear Energy Advocacy


[🎧 Play snip - 21sec️ (16:59 - 17:20)](https://share.snipd.com/snip/94767878-507a-46ae-a506-a51672e4c441)


**Nuclear Energy Advocacy**

- Advocate for increased nuclear energy development.
- It's the most environmentally friendly energy source, even better than solar.
- While solar will likely be widely adopted in the future, it's decades away and not as cheap as nuclear.
- Nuclear energy offers an immediate and sustainable energy solution.


#### 📚 Transcript
<details>
<summary>Click to expand</summary>
<blockquote><b>Gavin Baker</b><br/><br/>Nuclear is arguably just as environmentally friendly, done right and carefully. And it is here now. And so I just, yeah, I mean, that's, yeah.</blockquote><br/><blockquote><b>Jason Calacanis</b><br/><br/>It is unbelievable to watch, not exactly Moore's law, but this precipitous drop in the cost of just solar panels.</blockquote>
</details>



---


### [26:34] US Capital Markets


[🎧 Play snip - 1min️ (25:57 - 26:33)](https://share.snipd.com/snip/cb75233a-0d33-4b39-a92f-bd6ac5191f76)


**US Capital Markets**

- The U.S. has the best capital markets in the world, offering the most trusted equity and fixed income markets globally.
- These markets are vital for America's success, fostering trust and confidence among investors and businesses.
- Maintaining fairness and preventing insider information are crucial for preserving the integrity and strength of these markets.


#### 📚 Transcript
<details>
<summary>Click to expand</summary>
<blockquote><b>Gavin Baker</b><br/><br/>It is important to remember, we have the best capital markets in the world. You know, the U.S. Equity and fixed income markets are the most trusted places on earth. And we can always make them better. But just it is very, you know, you want to be very vigilant about keeping them fair and keeping out things like inside information, which makes people feel comfortable doing business Here, having investors have confidence in a company's financial statements. And those financial markets are one reason America is such a great country.</blockquote><br/><blockquote><b>Jason Calacanis</b><br/><br/>And Gavin, to put this in context, this</blockquote>
</details>



---


### [29:47] Bitcoin vs. USD


[🎧 Play snip - 1min️ (28:41 - 29:47)](https://share.snipd.com/snip/65852c59-5d4b-4a83-a8ad-87fa209d3cd2)


**Bitcoin vs. USD**

- David Friedberg agrees with Gavin Baker that Bitcoin is a potential threat to the US dollar.
- He points out the irony of Trump supporting Bitcoin while simultaneously opposing other alternative currencies.
- Friedberg believes that eventually, the US government will recognize Bitcoin as a threat.


#### 📚 Transcript
<details>
<summary>Click to expand</summary>
<blockquote><b>David Friedberg</b><br/><br/>Well, I definitely concur with Gavin. I think Bitcoin fundamentally is meant to be, supposed to be, ultimately will become a real threat to the US dollar. And it's kind of ironic that Trump had this declaration this week that he's going to put 100% tariff on all these BRICS nations that try to participate in an alternative currency to the US dollar, the greatest currency on earth, when he literally turns around and then says, we're going to support Bitcoin. It felt like the biggest irony of the week to me, because I do think Bitcoin is the big threat to the US dollar. And I do think that at some point, whether it's this administration or the next, they're going to wake up to that fact. And maybe the Bitcoin does, you know, the network state concept does emerge. And that's where we end up. But I do think we want to have and are going to have a strong federal government in the United States for quite some time, that's going to play an important role in everyone's lives here. And I don't know if you can really just say, let the dollar, you know, be supplanted by Bitcoin. Bitcoin seems to be a more of a safe haven asset and that seems to be the trade that it's store it should be kind of sort of value and alternative it's just going to take over gold what do you Think you think um the</blockquote>
</details>



---


### [38:25] Regulating Private Investments


[🎧 Play snip - 15sec️ (38:17 - 38:33)](https://share.snipd.com/snip/106ab833-c51e-43f2-926b-5d365e913153)


**Regulating Private Investments**

- If private companies were made available to ordinary investors like public companies, regulatory changes would be required.
- Private companies currently adhere to lower disclosure and reporting standards compared to their public counterparts.
- Public companies are subject to stricter standards for a reason, concerning financial transparency and data integrity.


#### 📚 Transcript
<details>
<summary>Click to expand</summary>
<blockquote><b>Gavin Baker</b><br/><br/>Generally pretty centrist in most things. So I agree with what a lot of David said. And I do think if you were to allow ordinary Americans to buy private companies that are held to a lower standard of disclosure and reporting the public companies, like somebody would Have to change.</blockquote>
</details>



---


### [48:23] Restricting China's Access to Advanced Compute


[🎧 Play snip - 1min️ (47:07 - 48:22)](https://share.snipd.com/snip/a243b418-31d6-40bc-befe-3fb96ab14062)


**Restricting China's Access to Advanced Compute**

- The US is restricting China's access to advanced computing and networking technology, like a "sophons" from the sci-fi novel *The Three-Body Problem*.
- Despite this, China remains close behind the US in AI development.
- New chips from NVIDIA, AMD, and Broadcom in the coming year might make it impossible for China to keep up.


#### 📚 Transcript
<details>
<summary>Click to expand</summary>
<blockquote><b>David Friedberg</b><br/><br/>How do you look at that market? You agree? I agree with everything Joe said.</blockquote><br/><blockquote><b>Gavin Baker</b><br/><br/>The only thing I would just add on China, what we are doing by restricting their access to advanced compute and advanced networking, if you have read or watched the three-body problem, America is unfolding a so-fond over China. Yeah.</blockquote><br/><blockquote><b>Joe Lonsdale</b><br/><br/>That's a great way to say it.</blockquote><br/><blockquote><b>Gavin Baker</b><br/><br/>I have been really impressed with, um, some of the Chinese models that have come out. And I think the risk to this strategy is necessity is the mother of invention. And despite this handicap, they're managing to stay just behind the leading edge of America, which is amazing. But, you know, NVIDIA's Blackwell chip comes out next year. You're going to have new chips from AMD, new A6 from Broadcom. And I think at that point, it is not going to be possible for them to keep up anymore.</blockquote><br/><blockquote><b>Jason Calacanis</b><br/><br/>So that's actually positive regulation and great, in your mind, foreign policy.</blockquote><br/><blockquote><b>Gavin Baker</b><br/><br/>It is very aggressive foreign policy yeah clearly you know that could have lots of unforeseen consequences yeah what do you think about the the rare earth trade restrictions</blockquote>
</details>



---


### [58:42] New Scaling Laws in AI


[🎧 Play snip - 1min️ (57:44 - 58:45)](https://share.snipd.com/snip/b768f335-42ae-403a-bc63-84c2916e55ac)


**New Scaling Laws in AI**

- Grok 3's large GPU cluster will provide data on whether scaling laws are breaking down or holding as models grow.
- There's a new scaling axis called 'test time compute' or 'inference scaling'.
-  Giving models more time to think about complex questions dramatically improves performance, similar to how humans tackle problems.


#### 📚 Transcript
<details>
<summary>Click to expand</summary>
<blockquote><b>Gavin Baker</b><br/><br/>The other question you raise, David, is very interesting. And by the way, we should note there is now a new axis of scaling. Some people call it test time compute. Some people call it inference scaling. And basically the way this works, you just think of these models as human. The more you speak to one of these models, the way you'd speak to your 17-year going off to take the SAT, the better it will do for you. As a human, if I ask you, David, what's two plus two, four flashes in your mind right away. If I ask you to, you know, unify a grand unified theory of physics that accounts for both quantum mechanics and relativistic physics, you will think for a lot longer. We have been, yeah, nobody knows. We have been giving these models the same amount of time to think no matter how complicated the question was. What we've now learned is if you let them think for longer about more complex questions, test time compute, you can dramatically improve their IQ. So we're just at the beginning of this new scaling law. But I think the question you raise on ROI is very good.</blockquote>
</details>



---


### [01:16:04] Health Insurance Business


[🎧 Play snip - 1min️ (01:15:22 - 01:16:11)](https://share.snipd.com/snip/a305583d-f49e-429e-8ec8-3652a91e6a55)


**Health Insurance Business**

- United Healthcare's medical loss ratio is ~85%, meaning they pay out 85 cents of every dollar collected in insurance premiums for claims.
- Health insurance, alongside auto insurance, is one of the most challenging businesses due to constant payouts and difficulty managing losses.
- There's a delicate balance between keeping health insurance affordable and managing the cost of medical claims. If all claims were paid, premiums would rise, making insurance unaffordable.


#### 📚 Transcript
<details>
<summary>Click to expand</summary>
<blockquote><b>David Friedberg</b><br/><br/>Yeah. United Healthcare's medical loss ratio is about 85%. So 85 cents of it. So 85 cents of every dollar they collect an insurance premium, they're paying out in claims. If you guys want to look at what the most egregious insurance industry in the world is, it's title insurance. And I'll give you the list of the rest. Travel insurance is pretty bad. They pay out like nothing. You were in the insurance business for a bit there. Yeah. Like, I mean, you know, health insurance is the hardest, one of the hardest besides auto insurance businesses to be in. You're paying out constantly. And there is a very difficult kind of process of managing losses, because the number of claims that comes in, it's very easy to suddenly pay everything out. And then your premium goes up, and then people can't afford the health insurance. So you're striking this balance of making health insurance affordable against the cost of medical claims.</blockquote>
</details>



---



"""

In [157]:
example_one_parsed = extract_episode_info(example_one)
example_one_parsed.model_dump()

{'episode_title': '#364 – Chris Voss: FBI Hostage Negotiator',
 'show': 'Lex Fridman Podcast',
 'owner_host': 'Lex Fridman',
 'episode_link': 'https://share.snipd.com/episode/afb1e4cc-e91d-482d-b55b-a7effcbdb029',
 'publish_date': '2023-03-10',
 'export_date': '2024-12-24T14:55',
 'show_notes': 'Chris Voss is a former FBI hostage and crisis negotiator and author of Never Split the Difference: Negotiating As If Your Life Depended On It. Please support this podcast by checking out our sponsors: Shopify :  https://shopify.com/lex  to get free trial; Indeed :  https://indeed.com/lex  to get $75 credit; InsideTracker :  https://insidetracker.com/lex  to get 20% off.',
 'ai_notes': None,
 'snips': [{'timestamp': '16:42',
   'snip_title': 'Articulate the Perspective of Others',
   'play_link': 'https://share.snipd.com/snip/5a706cfd-8c9f-4dd9-ac73-a0880c9fdc1a',
   'duration': '3min',
   'time_range': '15:25 - 18:52',
   'summary': "Empathy is the ability to understand and articulate the persp

In [158]:
import pandas as pd
responses = []
rejected_episodes = []

with open(file_path, 'r', encoding='utf-8') as f:
    content = f.read()

# Split content into episodes based on top-level headings (# )
episode_blocks = re.split(r'(?m)^# ', content)
episode_blocks = [block.strip() for block in episode_blocks if block.strip()]

all_snips = []

for idx, block in enumerate(episode_blocks):
    print(f"Processing Episode {idx + 1}")
    block = '# ' + block
    print(block[:50])
    try:
        extracted = extract_episode_info(block)
        responses.append(extracted)
    except Exception as e:
        print(e)
        rejected_episodes.append({"id": idx, "episode_md": block})
        continue

# make dataframe from responses and save to csv
df = pd.DataFrame(responses)
df.to_csv('responses_parsed_final.csv', index=False)
df_rejected = pd.DataFrame(rejected_episodes)
df_rejected.to_csv('rejected_episodes.csv', index=False)


Processing Episode 1
# #393 - Is History Repeating Itself?


<img src="
Processing Episode 2
# Dueling Presidential interviews, SpaceX’s big ca
Processing Episode 3
# Hurricane fallout, AlphaFold, Google breakup, Tr
Processing Episode 4
# DOGE unveils a roadmap, Unlocking GDP Growth, WW
Processing Episode 5
# New SEC Chair, Bitcoin, xAI Supercomputer, Unite
Processing Episode 6
# Grand Challenges in Healthcare AI


<img src="ht
Processing Episode 7
# #452 – Dario Amodei: Anthropic CEO on Claude, AG
Processing Episode 8
# DOGE kills its first bill, Zuck vs OpenAI, Googl
Processing Episode 9
# #392 - Technology & Culture


<img src="https://
Could not parse response content as the request was rejected by the content filter
Processing Episode 10
# How AI is Transforming Labor Markets


<img src=
Could not parse response content as the request was rejected by the content filter
Processing Episode 11
# Tesla's Road Ahead: The Bitter Lesson in Robotic
Processing Episode 12
# Best Of: Why the

In [168]:
results_destructured = []

for response in responses:
    for snip in response.snips:
        snip_data = snip.model_dump()
        # episode data excluding snips
        episode_data = response.model_dump()
        episode_data.pop('snips', None)
        # merge episode data and snip data
        merged_data = {**episode_data, **snip_data}
        results_destructured.append(merged_data)

results_parsed_df = pd.DataFrame(results_destructured)
results_parsed_df.to_csv('results_parsed_data.csv', index=False)
print("Unique episodes in results_parsed_df:")
print(len(results_parsed_df['episode_title'].unique()))
results_parsed_df


Unique episodes in results_parsed_df:
177


,episode_title,show,owner_host,episode_link,publish_date,export_date,show_notes,ai_notes,timestamp,snip_title,play_link,duration,time_range,summary,transcript_cleaned_of_md_and_html_tags,transcript_raw
0,#393 - Is History Repeating Itself?,Making Sense with Sam Harris - Subscriber Content,Making Sense with Sam Harris,https://share.snipd.com/episode/ba69c343-dda0-...,2024-11-26,2024-12-24T14:55,Sam Harris speaks with Simon Sebag Montefiore ...,None,08:00,Exceptional Period of Stability Ending,https://share.snipd.com/snip/fe100138-429c-4fe...,2min,05:46 - 08:05,The post-WWII era until recent times was excep...,Simon Seabag Montefiore: But I think the thing...,<blockquote><b>Simon Seabag Montefiore</b><br/...
1,#393 - Is History Repeating Itself?,Making Sense with Sam Harris - Subscriber Content,Making Sense with Sam Harris,https://share.snipd.com/episode/ba69c343-dda0-...,2024-11-26,2024-12-24T14:55,Sam Harris speaks with Simon Sebag Montefiore ...,None,41:21,Origin of Antisemitism,https://share.snipd.com/snip/54a1ffb1-1231-462...,2min,39:50 - 41:21,Antisemitism emerged when Christianity became ...,"Simon Seabag Montefiore: But you're right, fro...",<blockquote><b>Simon Seabag Montefiore</b><br/...
2,#393 - Is History Repeating Itself?,Making Sense with Sam Harris - Subscriber Content,Making Sense with Sam Harris,https://share.snipd.com/episode/ba69c343-dda0-...,2024-11-26,2024-12-24T14:55,Sam Harris speaks with Simon Sebag Montefiore ...,None,44:50,Jewish Otherness,https://share.snipd.com/snip/2b917d9c-dd6f-459...,1min,43:34 - 44:48,"This distinct identity defined Jews, making th...",Simon Seabag Montefiore: And I think it was be...,<blockquote><b>Simon Seabag Montefiore</b><br/...
3,#393 - Is History Repeating Itself?,Making Sense with Sam Harris - Subscriber Content,Making Sense with Sam Harris,https://share.snipd.com/episode/ba69c343-dda0-...,2024-11-26,2024-12-24T14:55,Sam Harris speaks with Simon Sebag Montefiore ...,None,49:42,Crusader Massacre in Jerusalem,https://share.snipd.com/snip/319c8640-bdaf-436...,1min,48:17 - 49:42,"During the First Crusade, a small group of cru...",Simon Seabag Montefiore: And then they fought ...,<blockquote><b>Simon Seabag Montefiore</b><br/...
4,#393 - Is History Repeating Itself?,Making Sense with Sam Harris - Subscriber Content,Making Sense with Sam Harris,https://share.snipd.com/episode/ba69c343-dda0-...,2024-11-26,2024-12-24T14:55,Sam Harris speaks with Simon Sebag Montefiore ...,None,51:15,The Crusades and October 7th,https://share.snipd.com/snip/30d1e5d0-b9ae-44d...,1min,49:47 - 51:14,The brutality of the 1099 Crusader massacre ec...,Simon Seabag Montefiore: So a chilling moment....,<blockquote><b>Simon Seabag Montefiore</b><br/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,#364 – Chris Voss: FBI Hostage Negotiator,Lex Fridman Podcast,Lex Fridman,https://share.snipd.com/episode/afb1e4cc-e91d-...,2023-03-10,2024-12-24T14:55,Chris Voss is a former FBI hostage and crisis ...,None,16:42,Articulate the Perspective of Others,https://share.snipd.com/snip/5a706cfd-8c9f-4dd...,3min,15:25 - 18:52,Empathy is the ability to understand and artic...,So Bob's definition of empathy said not agreei...,<blockquote><b>Speaker 1</b><br/><br/>So Bob's...
454,#57 David Deutsch - The Multiverse is Real,Within Reason,Alex O'Connor,https://share.snipd.com/episode/4a2c2c4a-505c-...,2024-03-05,2024-12-24T14:55,David Deutsch is a British physicist at the Un...,None,03:54,Interpreting Quantum Theory,https://share.snipd.com/snip/43a854f9-5058-4eb...,02:14 - 03:53,None,A small proportion of theoretical physicists e...,"Are perhaps 10% of theoretical physicists, but...",<blockquote><b>Speaker 1</b><br/><br/>Are perh...
455,#57 David Deutsch - The Multiverse is Real,Within Reason,Alex O'Connor,https://share.snipd.com/episode/4a2c2c4a-505c-...,2024-03-05,2024-12-24T14:55,David Deutsch is a British physicist at the Un...,None,15:28,Parallel Universes Indicated by Photon Behavior,https://shar

## Reprocess rejected episodes

In [194]:
print("Attempting to reprocess rejected episodes...")
reprocessed_responses = []

for rejected in rejected_episodes:
    try:
        print(f"Reprocessing episode {rejected['id']}")
        extracted = extract_episode_info(rejected['episode_md'])
        reprocessed_responses.append(extracted)
        rejected_episodes.remove(rejected)
    except Exception as e:
        print(f"Failed to process episode {rejected['id']}: {str(e)}")
        continue

if reprocessed_responses:
    print(f"Successfully reprocessed {len(reprocessed_responses)} episodes")
    # Add reprocessed responses to main results
    responses.extend(reprocessed_responses)
else:
    print("No episodes were successfully reprocessed")


Attempting to reprocess rejected episodes...
Reprocessing episode 20
Failed to process episode 20: Could not parse response content as the request was rejected by the content filter
Reprocessing episode 167
Failed to process episode 167: Could not parse response content as the request was rejected by the content filter
No episodes were successfully reprocessed


In [195]:
rejected_episodes

[{'id': 20,
  'episode_md': '# #395 - Intellectual Authority and Its Discontents\n\n\n<img src="https://wsrv.nl/?url=https%3A%2F%2Fassets.samharris.org%2Fimages%2Frss%2Fmaking-sense-logo.png&w=200&h=200" width="200" alt="Cover" />\n\n\n## Episode metadata\n- Episode title: #395 - Intellectual Authority and Its Discontents\n- Show: Making Sense with Sam Harris - Subscriber Content\n- Owner / Host: Making Sense with Sam Harris\n- Episode link: [open in Snipd](https://share.snipd.com/episode/5c5dd0bc-226b-4af6-97c5-8766b180347c)\n- Episode publish date: 2024-12-11\n<details>\n<summary>Show notes</summary>\n> Share this episode:  https://www.samharris.org/podcasts/making-sense-episodes/395-intellectual-authority-and-its-discontents <br/>> <br/>>  Sam Harris discusses the breakdown of trust in institutions, the nature of intellectual authority, the danger of bad incentives, the epidemic of conspiracy thinking and misinformation, Trump and Elon, and other topics.<br/>> <br/>>   <br/>> <br/>>

In [189]:
reprocessed_results_destructured = []

for response in reprocessed_responses:
    for snip in response.snips:
        snip_data = snip.model_dump()
        # episode data excluding snips
        episode_data = response.model_dump()
        episode_data.pop('snips', None)
        # merge episode data and snip data
        merged_data = {**episode_data, **snip_data}
        reprocessed_results_destructured.append(merged_data)

reprocessed_results_parsed_df = pd.DataFrame(reprocessed_results_destructured)
reprocessed_results_parsed_df.to_csv('reprocessed_results_parsed_data5.csv', index=False)
print("Unique episodes in reprocessed_results_parsed_df:")
print(len(reprocessed_results_parsed_df['episode_title'].unique()))
reprocessed_results_parsed_df

Unique episodes in reprocessed_results_parsed_df:
1


,episode_title,show,owner_host,episode_link,publish_date,export_date,show_notes,ai_notes,timestamp,snip_title,play_link,duration,time_range,summary,transcript_cleaned_of_md_and_html_tags,transcript_raw
0,The State of AI with Marc & Ben,a16z Podcast,a16z,https://share.snipd.com/episode/8ef10934-c249-...,2024-06-14,2024-12-24T14:55,"In this latest episode on the State of AI, Ben...",None,[11:45],Crafting Prompts for AI to Access Super Genius...,https://share.snipd.com/snip/e09a163c-3182-434...,2min,(10:12 - 11:48),Training AI models on average internet data re...,None,<details>\n<summary>Click to expand</summary>\...
1,The State of AI with Marc & Ben,a16z Podcast,a16z,https://share.snipd.com/episode/8ef10934-c249-...,2024-06-14,2024-12-24T14:55,"In this latest episode on the State of AI, Ben...",None,[31:40],The Jevons Paradox in Software Development,https://share.snipd.com/snip/0238c7eb-dfa2-4a2...,1min,(30:24 - 31:42),The Jevons Paradox is evident in software deve...,None,<details>\n<summary>Click to expand</summary>\...


## Analysis

In [208]:
# load reprocessed_results_parsed_data5.csv
results_parsed_df = pd.read_csv('results_parsed_data.csv')
reprocessed_results_parsed_df = pd.read_csv('reprocessed_results_parsed_data.csv')
reprocessed_results_parsed_df2 = pd.read_csv('reprocessed_results_parsed_data2.csv')
reprocessed_results_parsed_df3 = pd.read_csv('reprocessed_results_parsed_data3.csv')
reprocessed_results_parsed_df4 = pd.read_csv('reprocessed_results_parsed_data4.csv')
reprocessed_results_parsed_df5 = pd.read_csv('reprocessed_results_parsed_data5.csv')

# join them together verticalls
full_df = pd.concat([results_parsed_df, reprocessed_results_parsed_df, reprocessed_results_parsed_df2, reprocessed_results_parsed_df3, reprocessed_results_parsed_df4, reprocessed_results_parsed_df5], ignore_index=True)
full_df.describe()
full_df.to_csv('full_df.csv', index=False)